In [7]:
import numpy as np
import sunpy
import sunpy.map
from sunpy.coordinates import get_horizons_coord, propagate_with_solar_surface
import matplotlib.pyplot as plt
from scipy.interpolate import LinearNDInterpolator
import astropy
import astropy.units as u
from astropy.io import fits
import astropy.constants as const
from astropy.coordinates import SkyCoord
from glob import glob
import os
from watroo import wow
from sunraster.instr.spice import read_spice_l2_fits

In [ ]:
def interpolate_spice_map_to_target_wcs(spice_map, spice_coalign_wcs, spice_time,
                                        hri_map,
                                        spice_solarx_shift=spice_solarx_shift, 
                                        spice_solary_shift=spice_solary_shift):
    spice_nx = spice_nt = spice_map.data.shape[1]
    spice_map = spice_map.submap([0, 120]*u.pix, top_right=[spice_nx, 699]*u.pix)
    spice_ny = spice_map.data.shape[0]
    spice_pix_t, spice_pix_y, spice_pix_x = np.indices((1,*spice_map.data.shape))
    spice_world_coords = spice_coalign_wcs.pixel_to_world(spice_pix_x, spice_pix_y, spice_pix_t)[0][0,:,:]

    solar_orbiter_loc = np.flip(get_horizons_coord('solar orbiter',
                                                {'start':spice_time[-1],
                                                'stop':spice_time[0],
                                                'step':f'{spice_nt}'}))
    
    spice_pix_y_in_target_wcs = np.zeros((spice_ny, spice_nx))
    spice_pix_x_in_target_wcs = np.zeros((spice_ny, spice_nx))

    target_wcs = hri_map.wcs

    for ii in range(spice_nt):
        spice_world_coord_t = SkyCoord(spice_world_coords[:,ii].Tx.to(u.arcsec) + spice_solarx_shift[ii]*u.arcsec, 
                                  spice_world_coords[:,ii].Ty.to(u.arcsec) + spice_solary_shift[ii]*u.arcsec,
                                  frame='helioprojective',obstime=spice_time[ii], 
                                  observer=solar_orbiter_loc[ii], 
                                  rsun=hri_map.meta['rsun_ref']*u.m,)
        
        with propagate_with_solar_surface(rotation_model='rigid'):
            spice_pix_x_in_target_wcs[:,ii], spice_pix_y_in_target_wcs[:,ii] = target_wcs.world_to_pixel(spice_world_coord_t)

    hri_map_pix_y, hri_map_pix_x = np.indices(hri_map.data.shape)

    spice_map_interpolator = LinearNDInterpolator((spice_pix_x_in_target_wcs.flatten(), spice_pix_y_in_target_wcs.flatten()), spice_map.data.flatten())

    spice_map_interpolated = spice_map_interpolator(hri_map_pix_x, hri_map_pix_y)
    spice_map_interpolated = sunpy.map.Map(spice_map_interpolated, target_wcs)
    spice_map_interpolated.meta['rsun_ref'] = hri_map.meta['rsun_ref']
    return spice_map_interpolated

In [ ]:
def get_saffron_map(saffron_file_dir, spice_cube, spice_time, hri_map, 
                    spice_solarx_shift, spice_solary_shift):
    spice_coalign_wcs = spice_cube['Ne VIII 770 - Peak'].wcs.dropaxis(2)[:,120:700,:]

    saffron_NeVIII_file_path = glob(os.path.join(saffron_file_dir, '*770.42-ne_8*.fits'))[0]
    saffron_NIV_file_path = glob(os.path.join(saffron_file_dir, '*765.15-n_4*.fits'))[0]

    saffron_map = sunpy.map.Map(saffron_NIV_file_path)
    saffron_int_map = saffron_map[0]
    saffron_wid_map = saffron_map[2]
    saffron_int_map = sunpy.map.Map(saffron_int_map.data*saffron_wid_map.data*np.sqrt(np.pi), saffron_int_map.meta)
    saffron_int_map = interpolate_spice_map_to_target_wcs(saffron_int_map, spice_coalign_wcs, spice_time, hri_map, 
                                                         spice_solarx_shift, spice_solary_shift)
    
    saffron_map = sunpy.map.Map(saffron_NeVIII_file_path)
    saffron_vel_map = saffron_map[1]

    saffron_velmap_data = saffron_vel_map.data.copy()
    saffron_velmap_data = (saffron_velmap_data/np.nanmedian(saffron_velmap_data) - 1)*const.c.to_value(u.km/u.s)
    saffron_velmap_median = np.nanmedian(saffron_velmap_data[120:699,:], axis=0)
    saffron_velmap_median_fit_param = np.polyfit(np.arange(saffron_velmap_median.shape[0]), saffron_velmap_median, 1)
    saffron_velmap_data = saffron_velmap_data - np.polyval(saffron_velmap_median_fit_param, np.arange(saffron_velmap_median.shape[0]))[np.newaxis,:]
    saffron_vel_map = sunpy.map.Map(saffron_velmap_data, saffron_map[1].meta)

    saffron_vel_map = interpolate_spice_map_to_target_wcs(saffron_vel_map, spice_coalign_wcs, spice_time, hri_map,
                                                            spice_solarx_shift, spice_solary_shift)

    return saffron_int_map, saffron_vel_map
    

In [ ]:
def get_hri_hrt_spice(hri_file_path, hrt_file_path, spice_file_path,
                      spice_saffron_file_dir,bottom_left_coord, top_right_coord,
                      hri_Txshift=0*u.arcsec, hri_Tyshift=0*u.arcsec):
    hri_map = sunpy.map.Map(hri_file_path)
    hri_map = hri_map.shift_reference_pixel(hri_Txshift, hri_Tyshift)
    hri_map = hri_map.submap(bottom_left_coord, top_right=top_right_coord)

    hrt_map = sunpy.map.Map(hrt_file_path)
    hrt_map = hrt_map.submap(bottom_left_coord, top_right=top_right_coord)
    
    spice_cube = read_spice_l2_fits(spice_file_path)
    with fits.open(spice_file_path) as hdul:
        spice_solarx_shift = hdul[-2].data.copy()
        spice_solary_shift = hdul[-1].data.copy()

    spice_time = spice_cube['Ne VIII 770 - Peak'].time[0]
    
    return hri_map, hrt_map, spice_map

In [ ]:
def get_saffron_map(saffron_dir, saffron_filename, spice_cube_name, spice_time, hri_map, velmap=False):
    saffron_files = glob(os.path.join(saffron_dir, saffron_filename))

    spice_coalign_wcs = spice_coalign_cube[spice_cube_name].wcs.dropaxis(2)[:,120:700,:]

    saffron_intmaps = []
    if velmap:
        saffron_velmaps = []

    for saffron_file in saffron_files:
        saffron_map = sunpy.map.Map(saffron_file)
        saffron_intmap = saffron_map[0]
        saffron_widmap = saffron_map[2]
        saffron_intmap = sunpy.map.Map(saffron_intmap.data*saffron_widmap.data*np.sqrt(np.pi), saffron_intmap.meta)
        saffron_intmap = interpolate_spice_map_to_target_wcs(saffron_intmap, spice_coalign_wcs, spice_time, hri_map)

        saffron_intmap.plot_settings['norm'] = ImageNormalize(vmin=np.nanpercentile(saffron_intmap.data, 0.5),
                                                                vmax=np.nanpercentile(saffron_intmap.data, 99.5),
                                                                stretch=AsinhStretch())
        saffron_intmaps.append(saffron_intmap)


        if velmap:
            saffron_velmap = saffron_map[1]

            saffron_velmap_data = saffron_velmap.data.copy()
            saffron_velmap_data = (saffron_velmap_data/np.nanmedian(saffron_velmap_data) - 1)*const.c.to_value(u.km/u.s)
            saffron_velmap_median = np.nanmedian(saffron_velmap_data[120:699,:], axis=0)
            saffron_velmap_median_fit_param = np.polyfit(np.arange(saffron_velmap_median.shape[0]), saffron_velmap_median, 1)
            saffron_velmap_data = saffron_velmap_data - np.polyval(saffron_velmap_median_fit_param, np.arange(saffron_velmap_median.shape[0]))[np.newaxis,:]
            saffron_velmap = sunpy.map.Map(saffron_velmap_data, saffron_map[1].meta)

            saffron_velmap = interpolate_spice_map_to_target_wcs(saffron_velmap, spice_coalign_wcs, spice_time, hri_map)
            saffron_velmap.plot_settings['norm'] = ImageNormalize(vmin=-40, vmax=40)
            saffron_velmaps.append(saffron_velmap)

    if velmap:
        if len(saffron_files) == 1:
            return saffron_intmaps[0], saffron_velmaps[0]
        else:
            return saffron_intmaps, saffron_velmaps
    else:
        if len(saffron_files) == 1:
            return saffron_intmaps[0]
        else:
            return saffron_intmaps